In [1]:
# Import libraries
import pandas as pd
import sys
import os

In [2]:
# Get the current working directory
current_dir = os.getcwd()
# Move up one level from the current directory
parent_dir = os.path.dirname(current_dir)
# Change directory into data directory
data_dir = os.path.join(parent_dir, 'data')

In [3]:
sys.path.append(parent_dir)

In [4]:
import db_config as cfg

# 1. Extract

In [5]:
df_product = pd.read_csv(os.path.join(data_dir, 'products.csv'), index_col=None)

In [6]:
df_product.head()

,item_desc,corp_item_brand_name,pim_item_class_desc,pim_item_sub_class_desc,state,flavor,pim_tasting_notes,alcohol_percentage,sweetness_level,bitterness_level,...,body,serving_temperature,vintage_year,grape_variety,region,price,food_pairing,aroma,mouthfeel,finish
0,-196 CKTL VOD A(DL/G/P) CAN 3/8PK,-196,COCKTAILS,COCKTAILS-OTHER,TX,OTHER,NaN,13.417170,10,3,...,Full,42.930000,2008,Chardonnay,Tuscany,53.103866,Pasta,Fruity,Rough,Long
1,-196 CKTL VOD PEACH 12 CAN 6/4PK,-196,COCKTAILS,COCKTAILS-OTHER,TX,PEACH,NaN,8.623515,9,3,...,Medium,59.049843,2018,Cabernet Sauvignon,Sonoma,94.475113,Pasta,Floral,Silky,Long
2,-196 CKTL VOD DBL LEM 12 CAN 6/4PK,-196,COCKTAILS,COCKTAILS-OTHER,TX,LEMON,Fresh lemon peel. Tart and light sweet with ch...,10.249378,2,8,...,Medium,49.068042,2003,Cabernet Sauvignon,Sonoma,85.019099,Cheese,Spicy,Silky,Medium
3,-196 CKTL VOD GRFRUIT 12 CAN 6/4P,-196,COCKTAILS,COCKTAILS-OTHER,TX,GRAPEFRUIT,NaN,6.012468,5,4,...,Medium,51.566561,2013,Cabernet Sauvignon,Rioja,87.519005,Cheese,Spicy,Silky,Short
4,10 CANE RUM 80,10 CANE,RUM,GOLD RUM,TX,NaN,NaN,10.071682,5,8,...,Full,61.513052,2016,Merlot,Napa Valley,89.105149,Pasta,Fruity,Rough,Short


# 2. Transform

In [7]:
df_product.dtypes

item_desc                   object
corp_item_brand_name        object
pim_item_class_desc         object
pim_item_sub_class_desc     object
state                       object
flavor                      object
pim_tasting_notes           object
alcohol_percentage         float64
sweetness_level              int64
bitterness_level             int64
acidity_level                int64
tannin_level                 int64
body                        object
serving_temperature        float64
vintage_year                 int64
grape_variety               object
region                      object
price                      float64
food_pairing                object
aroma                       object
mouthfeel                   object
finish                      object
dtype: object

# 3. Load

In [8]:
import pyodbc

In [9]:
#test connection
conn_str = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    F'SERVER={cfg.SERVER_NAME};'
    F'DATABASE={cfg.DATABASE_NAME};'
    F'UID={cfg.USER_ID};'
    F'PWD={cfg.USER_PASSWORD};'
)

In [10]:
def get_column_types(df):
    # Map Pandas data types to SQL Server data types
    type_map = {
        'object': 'NVARCHAR(MAX)',
        'int64': 'BIGINT',
        'float64': 'DECIMAL(18, 5)',  # Use DECIMAL with precision and scale
        'datetime64[ns]': 'DATETIME2',
        'bool': 'BIT'
    }
    return [type_map.get(str(dt), 'NVARCHAR(MAX)') for dt in df.dtypes]


In [11]:
def load_table(df_product, table_name, schema):
    # Define the connection string
    conn_str = (
        'DRIVER={ODBC Driver 17 for SQL Server};'
        F'SERVER={cfg.SERVER_NAME};'
        F'DATABASE={cfg.DATABASE_NAME};'
        F'UID={cfg.USER_ID};'
        F'PWD={cfg.USER_PASSWORD};'
    )

    # Create a pyodbc connection using the connection string
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()


    # Drop the table if it already exists
    cursor.execute(f"IF OBJECT_ID('{schema_name}.{table_name}', 'U') IS NOT NULL DROP TABLE {schema_name}.{table_name}")
    conn.commit()

    # Create the table schema
    columns = ', '.join([f'{col} {dtype}' for col, dtype in zip(df.columns, get_column_types(df))])
    create_table_sql = f"CREATE TABLE {schema_name}.{table_name} ({columns})"
    cursor.execute(create_table_sql)
    conn.commit()

    # Clean up the float columns in the DataFrame
    for col in df.select_dtypes(include=['float']):
        df[col] = df[col].round(5)  # Round to 5 decimal places

    # Ensure there are no NaN or None values in numeric columns
    df.fillna(0, inplace=True)  # You can adjust the fill value as necessary

    # Insert the data into the table
    insert_sql = f"INSERT INTO {schema_name}.{table_name} VALUES ({','.join(['?'] * len(df.columns))})"
    for _, row in df.iterrows():
        cursor.execute(insert_sql, *row.tolist())
    conn.commit()

    cursor.close()
    conn.close()
    print(f"DataFrame loaded successfully into {schema_name}.{table_name}")


In [12]:
load_table(df_product, 'products', 'dbo')

NameError: name 'schema_name' is not defined